# Qiskit device models

In order to explore what is feasible on IBM devices, you can now use tangelo to explore the types of errors one might expect when using particular IBM devices. These can be accessed by using the branch `qiskit-device-noise`.

The devices available are in `qiskit.test.mock`

In [1]:
# List all available Fake devices
from qiskit.test import mock
device_list = [func for func in dir(mock) if callable(getattr(mock, func))]
print(device_list)

['Fake1Q', 'FakeAlmaden', 'FakeArmonk', 'FakeAthens', 'FakeBackend', 'FakeBelem', 'FakeBoeblingen', 'FakeBogota', 'FakeBurlington', 'FakeCambridge', 'FakeCambridgeAlternativeBasis', 'FakeCasablanca', 'FakeEssex', 'FakeGuadalupe', 'FakeJob', 'FakeJohannesburg', 'FakeLegacyAlmaden', 'FakeLegacyArmonk', 'FakeLegacyAthens', 'FakeLegacyBackend', 'FakeLegacyBelem', 'FakeLegacyBoeblingen', 'FakeLegacyBogota', 'FakeLegacyBurlington', 'FakeLegacyCambridge', 'FakeLegacyCambridgeAlternativeBasis', 'FakeLegacyCasablanca', 'FakeLegacyEssex', 'FakeLegacyJob', 'FakeLegacyJohannesburg', 'FakeLegacyLima', 'FakeLegacyLondon', 'FakeLegacyManhattan', 'FakeLegacyMelbourne', 'FakeLegacyMontreal', 'FakeLegacyMumbai', 'FakeLegacyOurense', 'FakeLegacyParis', 'FakeLegacyPoughkeepsie', 'FakeLegacyProvider', 'FakeLegacyQuito', 'FakeLegacyRochester', 'FakeLegacyRome', 'FakeLegacyRueschlikon', 'FakeLegacySantiago', 'FakeLegacySingapore', 'FakeLegacySydney', 'FakeLegacyTenerife', 'FakeLegacyTokyo', 'FakeLegacyToront

In [2]:
from tangelo.linq import Circuit, Gate, Simulator
from tangelo.linq.noisy_simulation import NoiseModel

Below is an example for `"FakeMontreal"`. For any of the devices listed above, instantiate 
```python
sim_device = Simulator(target="qiskit_device", noise_model=NoiseModel("Device_Name"))
```

In [3]:
# Construct Quantum circuit
gates = [Gate('H', 0), Gate('CNOT', 0, 1), Gate('CNOT', 1, 2)]
circuit = Circuit(gates)

sim_ideal = Simulator('qiskit', n_shots=10000)
sim_montreal = Simulator('qiskit_device', n_shots=10000, noise_model=NoiseModel("FakeMontreal"))

freqs_ideal, _ = sim_ideal.simulate(circuit)
freqs_montreal, _ = sim_montreal.simulate(circuit)

print(freqs_ideal)
print(freqs_montreal)


{'100': 0.4969, '000': 0.5031}
{'101': 0.007, '011': 0.0003, '001': 0.0057, '100': 0.469, '000': 0.4691, '010': 0.026, '110': 0.0229}


## Choosing Qubits
Some qubits on the device have better properties. To test which qubits are best, we can assign `qubits_to_use`.

In [4]:
from tangelo.algorithms.variational import VQESolver
from tangelo.molecule_library import mol_H2_sto3g
from tangelo.algorithms.variational import BuiltInAnsatze

In [5]:
vqe_solver = VQESolver({"molecule": mol_H2_sto3g, "up_then_down": True, "ansatz": BuiltInAnsatze.HEA, "qubit_mapping": "scbk", "ansatz_options": {"n_layers": 1, "rot_type": "real"}})
vqe_solver.build()
print(vqe_solver.simulate())

-1.137267172271324


To use Montreal device qubits 0,1. We can obtaint the expectation value as normal.

In [6]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit)

-1.0627380844095529

Looking at https://quantum-computing.ibm.com/services?services=systems, it appears that 19,20 are better qubits to use.

In [7]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit, qubits_to_use=[19,20])

-1.0913967693329196

and indeed, you do get more accurate results using 19 and 20 compared to 0 and 1. 

Things to note. 
1. The qubits in the circuit must all be used and be in the `range(n_qubits)`. This can be done with the function `circuit.trim_qubits()`.
2. You can reorder as you like with `qubits_to_list=[25,24,16,19]`
3. The length of qubits_to_list must equal the number of qubits

## Choosing optimization level
Optimization level can be 0, 1, 2, 3. 3 being the most aggressive.

In [8]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit, qubits_to_use=[19,20], opt_level=3)

-1.0880423431860027

It does not seem to do much here

## measurement_error_mitigation
This will take extra time as it reruns the calibration circuit for each of the $2^{n_{qubits}}$ labels for 10,000 shots for each time. It does appear to work quite well here though.

In [9]:
freqs_montreal_mitt, _ = sim_montreal.simulate(circuit, meas_mitt=True)

print(freqs_montreal)
print(freqs_montreal_mitt)
print(freqs_ideal)

{'101': 0.007, '011': 0.0003, '001': 0.0057, '100': 0.469, '000': 0.4691, '010': 0.026, '110': 0.0229}
{'000': 0.5012422594931043, '100': 0.49649910395119895, '010': 0.0022586365556965565, '001': 1.283745462363206e-16, '101': 2.4089842533964176e-16, '111': 1.6061674559808558e-17}
{'100': 0.4969, '000': 0.5031}


In [10]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit, qubits_to_use=[19,20], opt_level=3, meas_mitt=True)

-1.129038350031206